<a href="https://colab.research.google.com/github/Mang9ae/AI_Aplication/blob/main/10%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate

In [ ]:
import os
os.environ['HF_TOKEN'] = "발급 받은 토큰"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#모델 ID 에 맞는 토크나이저 가져오기
tokenizer = AutoTokenizer.from_pretrained(model_id)
#모델 ID 에 해당하는 model 로딩하기
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",)

In [ ]:
def generate_response(system_message, user_message):
  #대화 맥락(chat history)
  messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
  ]

  #전달된 message 에 대해서 프롬프트 자동 생성
  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt").to(model.device)

  #응답 종료 기준(종료 토큰), end of turn
  terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  #텍스트 생성을 위한 설정 및 생성
  #max_new_tokens: 최대 생성 토큰 수
  #temperature(sampling temperature): 낮을수록 더 결정적인 응답, 높으면 다양성 증가
  #top_p: 확률 누적 합이 p 이하인 후보군에서 샘플링
  outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
!wget https://raw.githubusercontent.com/jskimn/aiservice_lesson/main/003_llama3/NvidiaSmiMonitor.py
from NvidiaSmiMonitor import NvidiaSmiMonitor
import sys
import time

monitor = NvidiaSmiMonitor(interval=60)  # 2초마다 실행
monitor.start()

# 10초 동안 실행 후 종료
#time.sleep(10)

In [ ]:
response = generate_response(system_message="",
                              user_message="한국어로 대답해줘. 배화여대에 대해서 말해줘")
print(response)


monitor.stop()
monitor.join()
print("monitor---------종료")